## Introduction

이 튜토리얼은 **Generative Adversarial Networks**(**GAN**)을 사용한 음악 생성에 대한 간략한 소개입니다.

이 튜토리얼의 목표는 모델이 싱글 트랙 입력 멜로디에 반주를 추가하는 방법을 배우도록 바흐(Bach) 컴포지션의 데이터셋을 사용하여 머신 러닝 모델을 학습하는 것입니다. 다시 말해, 사용자가 "twinkle twinkle little star"와 같은 노래의 싱글 피아노 트랙을 입력하는 경우, GAN 모델은 3개의 다른 피아노 트랙을 추가하여 음악 사운드를 보다 바흐 스타일과 가깝게 연주합니다.

제안된 알고리즘은 두 개의 경쟁 네트워크인 generator와 critic(discriminator)로 구성됩니다. Generator는 학습된 데이터셋의 분포와 유사한 새로운 합성 데이터를 생성하는 방법을 배우는 딥 뉴럴 네트워크입니다. Critic은 실제 데이터와 합성 데이터를 구별하도록 학습된 또 다른 딥 뉴럴 네트워크입니다. Generator와 critic은 교대 주기로 학습되어 generator는 점점 더 현실적인 데이터(이 사용 사례에서는 바흐와 같은 음악)를 생성하는 법을 배우고, critic은 실제 데이터(바흐 음악)와 합성 데이터를 구별하는 법을 더 잘 배우게 됩니다.

결과적으로, generator가 생성한 음악의 품질은 시간이 지날수록 더욱 현실감 있게 됩니다.

![High level WGAN-GP architecture](images/dgan.png "WGAN-GP architecture")

## Dependencies

먼저, 본 튜토리얼에서 사용할 모든 파이썬 패키지들을 가져 오겠습니다.

In [ ]:
# Copyright 2019 Amazon.com, Inc. or its affiliates. All Rights Reserved.

# Permission is hereby granted, free of charge, to any person obtaining a copy of
# this software and associated documentation files (the "Software"), to deal in
# the Software without restriction, including without limitation the rights to
# use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of
# the Software, and to permit persons to whom the Software is furnished to do so.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS
# FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR
# COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER
# IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
# CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.


# Create the environment
!conda update --all --y 
!pip install tensorflow-gpu==1.14.0
!pip install numpy==1.16.4
!pip install pretty_midi
!pip install pypianoroll
!pip install music21
!pip install seaborn
!pip install --ignore-installed moviepy

In [ ]:
# IMPORTS
import os 
import numpy as np
from PIL import Image
import logging
import pypianoroll
import scipy.stats
import pickle
import music21
from IPython import display
import matplotlib.pyplot as plt

# Configure Tensorflow
import tensorflow as tf
print(tf.__version__)
tf.logging.set_verbosity(tf.logging.ERROR)
tf.enable_eager_execution()

# Use this command to make a subset of GPUS visible to the jupyter notebook.
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"

# Utils library for plotting, loading and saving midi among other functions
from utils import display_utils, metrics_utils, path_utils, inference_utils, midi_utils

LOGGER = logging.getLogger("gan.train")
%matplotlib inline

## Configuration

데이터셋을 검색하고 실험 결과를 저장하기 위한 경로를 설정합니다.

In [ ]:
root_dir = './Experiments'

# Directory to save checkpoints
model_dir = os.path.join(root_dir,'2Bar')    # JSP: 229, Bach: 19199

# Directory to save pianorolls during training
train_dir = os.path.join(model_dir, 'train')

# Directory to save checkpoint generated during training
check_dir = os.path.join(model_dir, 'preload')

# Directory to save midi during training
sample_dir = os.path.join(model_dir, 'sample')

# Directory to save samples generated during inference
eval_dir = os.path.join(model_dir, 'eval')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(eval_dir, exist_ok=True)
os.makedirs(sample_dir, exist_ok=True)


## Data Preparation

### 데이터셋 요약

이 튜토리얼에서는 229개의 코랄(chorale) snippet들로 구성된 [`JSB-Chorales-dataset`](http://www-etud.iro.umontreal.ca/~boulanni/icml2012)을 사용합니다. 코랄은 찬송가로 보통 싱글 멜로디를 연주하는 싱글 음색(voice)과 하모니를 제공하는 3개의 하위 음색으로 노래됩니다. 이 데이터셋에서 이 음색은 4개의 피아노 트랙으로 표시됩니다.

이 데이터셋의 노래를 한 번 들어 보겠습니다.

In [ ]:
display_utils.playmidi('./original_midi/MIDI-0.mid')

### 데이터 포맷 - piano roll

본 튜토리얼의 실습을 위해, JSB-Choales 데이터셋의 음악을 piano roll 포맷으로 표현해 보겠습니다.

**Piano roll**은 음악을 머신 러닝 알고리즘들의 입력에 적용할 수 있게 변환한 이산적인(discrete) 표현이며, 가로 축에 "시간(Time)", 세로 축에 "피치(Pitch)"가 있는 2차원 격자입니다. 이 그리드의 특정 셀에서 1 또는 0은 해당 피치에 대해 음이 연주되었는지 여부를 나타냅니다.

데이터셋에서 piano roll 몇 개를 살펴 보겠습니다. 이 예시에서, 싱글 piano roll 트랙은 32개의 이산적인 타임스텝(time step)과 128개의 피치를 갖습니다. 여기에는 4개의 piano roll들이 있는데, 각 piano roll은 노래에서 별도의 피아노 트랙을 나타냅니다.

<img src="images/pianoroll2.png" alt="Dataset summary" width="800">

이 표현이 이미지(image)와 비슷하게 보일 수 있습니다. 일련의 음표가 사람들이 음악을 보는 자연스러운 방법인 경우가 많지만, 많은 현대 머신 러닝 모델들은 음악을 이미지로 취급하고 컴퓨터 비전 영역 내의 기존 기술을 활용합니다. 여러분은 이 튜토리얼의 뒷부분에서 Deepcomposer 아키텍처에 사용된 이러한 기술들을 보게 될 것입니다.

**왜 32개의 타임 스텝인가요?**

이 튜토리얼에서는 JSB-Chorales 데이터셋의 각 노래에서 2개의 non-empty bar (https://en.wikipedia.org/wiki/Bar_(music)) 를 샘플링합니다. **Bar** (또는 **measure**)는 구성 단위이며 특정 데이터셋의 노래에 대한 4개의 비트(beat)를 포함합니다 (우리의 노래는 모두 4/4 time입니다).

비트(beat) 당 4개의 타임 스텝 해상도를 사용하면 이 데이터셋의 음악적 세부 사항을 충분히 포착할 수 있으며, 이를 수식으로 표현하면 아래와 같습니다.

$$ \frac{4\;timesteps}{1\;beat} * \frac{4\;beats}{1\;bar} * \frac{2\;bars}{1} = 32\;timesteps $$

이제 데이터셋을 numpy 배열로 로드하겠습니다. 튜토리얼에서 사용할 데이터셋은 4개 트랙의 229개 샘플들로 구성됩니다(모든 트랙은 피아노입니다). 각 샘플은 노래의 32개 타임 스텝 snippet이므로 데이터셋의 크기는 다음과 같습니다.<br>
(num_samples, time_steps, pitch_range, tracks) = (229, 32, 128, 4).

In [ ]:
training_data = np.load('./dataset/train.npy')
print(training_data.shape)

모델에 공급(feed into)할 데이터 샘플을 보도록 하겠습니다. 4개의 그래프는 4개의 트랙을 나타냅니다.

In [ ]:
display_utils.show_pianoroll(training_data)

### 데이터 로드

이제, numpy 배열에서 Tensorflow dataset 객체를 만들어 모델에 공급(feed into)해 보겠습니다. Dataset 객체는 모델에 데이터 배치를 공급하는 데 도움이 됩니다. 배치는 가중치가 업데이트되기 전에 딥 러닝 네트워크를 통해 전달되는 데이터의 하위 집합입니다. 학습 환경에서 전체 데이터셋을 한 번에 메모리에 로드하지 못할 수 있으므로, 대부분의 학습 시나리오에서 데이터 배치가 필요합니다.

In [ ]:
#Number of input data samples in a batch
BATCH_SIZE = 64

#Shuffle buffer size for shuffling data
SHUFFLE_BUFFER_SIZE = 1000

#Preloads PREFETCH_SIZE batches so that there is no idle time between batches
PREFETCH_SIZE = 4

In [ ]:
def prepare_dataset(filename):
    
    """Load the samples used for training."""
    
    data = np.load(filename)
    data = np.asarray(data, dtype=np.float32)  # {-1, 1}

    print('data shape = {}'.format(data.shape))

    dataset = tf.data.Dataset.from_tensor_slices(data)
    dataset = dataset.shuffle(SHUFFLE_BUFFER_SIZE).repeat()
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    dataset = dataset.prefetch(PREFETCH_SIZE)

    return dataset 

dataset = prepare_dataset('./dataset/train.npy')

## Model architecture
이 섹션에서는 제안된 GAN의 아키텍처를 살펴 보겠습니다.

이 모델은 generator와 critic의 두 가지 네트워크로 구성됩니다. 이 두 네트워크는 다음과 같이 긴밀한(tight) 루프로 작동합니다.

* Generator :
    1. Generator는 싱글 트랙 piano roll(멜로디) 배치를 입력으로 취하고, 각 입력 음악 트랙에 반주를 추가하여 출력으로 멀티 트랙 piano roll 배치를 생성합니다.
    2. 그러면 critic은 생성된 음악 트랙을 가져 와서 학습 데이터셋에 존재하는 실제 데이터와 얼마나 멀리 떨어져 있는지 예측합니다.
    3. Critic으로부터의 이 피드백은 가중치를 업데이트하기 위해 generator에서 사용됩니다.
- Critic : Generator가 critic의 피드백을 사용하여 더 좋은 음악 반주를 만드는 데 익숙해 짐에 따라, critic도 재학습이 필요합니다.
    1. Generator에서 방금 생성한 음악 트랙을 가짜 입력으로, 원래 데이터셋과 동일한 수의 노래들을 실제 입력으로 하여 비평가를 학습시킵니다.
* 첫 에포크(epoch)에 대한 critic부터 시작하여, 모델이 수렴하고 보다 사실적인 음악을 생성할 때까지 이 두 네트워크를 교대로 전환합니다.

음악을 생성하기 위해, **Gradient Penalty가 있는 Wasserstein GAN** (또는 **WGAN-GP**)이라는 특별한 유형의 GAN을 사용합니다. WGAN-GP의 기본 아키텍처는 GAN의 vanilla 변형과 매우 유사하지만 WGAN-GP는 vanishing gradient 문제 및 mode collapse와 같이 GAN에서 일반적으로 보이는 결함들을 극복하는 데 도움이 됩니다 (자세한 내용은 appendix을 참조하세요).

참고로, "critic" 네트워크는 vanilla GAN의 더 일반적인 맥락에서, 일반적으로 "discriminator" 네트워크라고 불립니다.

### Generator

이 generator는 U-Net 아키텍처(컴퓨터 비전 도메인에서 널리 사용되는 유명한 CNN)를 채택한 것으로, 싱글 트랙 음악 데이터(piano roll롤 이미지로 표현)를 비교적 저차원 "latent 공간"에 매핑하는 "인코더(encoder)"와 latent 공간을 다시 멀티 트랙 음악 데이터에 매핑하는 "디코더(decoder)"로 구성되어 있습니다.

Generator의 입력 데이터는 다음과 같습니다.

**싱글 트랙 piano roll 입력**: 
(32, 128, 1) => (TimeStep, NumPitches, NumTracks) 크기의 싱글 멜로디 트랙이 generator에 대한 입력으로 전달됩니다.

**Latent noise 벡터**: 차원 (2, 8, 512)의 latent noise 벡터 $z$도 입력으로 전달되며, 이는 동일한 입력이 전달되는 경우라도 generator에 의해 생성된 각 출력이 고유한 특징이 있는지 확인하는 역할을 합니다.

참고로, 아래 그림에서 왼쪽에 있는 generator의 인코더 레이어와 오른쪽에 있는 디코더 레이어가 연결되어 U자 모양을 만들기에, 이 아키텍처에 U-Net이라는 이름이 붙여졌습니다.

<img src="images/dgen.png" alt="Generator architecture" width="800">

이 구현에서는 `_conv2d`와 `_deconv2d`를 결합하여, 간단한 4계층 U-Net 아키텍처를 따라 generator를 빌드합니다. 여기서 `_conv2d`는 contracting path를 구성하고 `_deconv2d`는 expansive path를 형성합니다.

In [ ]:
def _conv2d(layer_input, filters, f_size=4, bn=True):
    """Generator Basic Downsampling Block"""
    d = tf.keras.layers.Conv2D(filters, kernel_size=f_size, strides=2,
                               padding='same')(layer_input)
    d = tf.keras.layers.LeakyReLU(alpha=0.2)(d)
    if bn:
        d = tf.keras.layers.BatchNormalization(momentum=0.8)(d)
    return d


def _deconv2d(layer_input, pre_input, filters, f_size=4, dropout_rate=0):
    """Generator Basic Upsampling Block"""
    u = tf.keras.layers.UpSampling2D(size=2)(layer_input)
    u = tf.keras.layers.Conv2D(filters, kernel_size=f_size, strides=1,
                               padding='same')(u)
    u = tf.keras.layers.BatchNormalization(momentum=0.8)(u)
    u = tf.keras.layers.ReLU()(u)

    if dropout_rate:
        u = tf.keras.layers.Dropout(dropout_rate)(u)
        
    u = tf.keras.layers.Concatenate()([u, pre_input])
    return u

    
def build_generator(condition_input_shape=(32, 128, 1), filters=64,
                    instruments=4, latent_shape=(2, 8, 512)):
    """Buld Generator"""
    c_input = tf.keras.layers.Input(shape=condition_input_shape)
    z_input = tf.keras.layers.Input(shape=latent_shape)

    d1 = _conv2d(c_input, filters, bn=False)
    d2 = _conv2d(d1, filters * 2)
    d3 = _conv2d(d2, filters * 4)
    d4 = _conv2d(d3, filters * 8)

    d4 = tf.keras.layers.Concatenate(axis=-1)([d4, z_input])

    u4 = _deconv2d(d4, d3, filters * 4)
    u5 = _deconv2d(u4, d2, filters * 2)
    u6 = _deconv2d(u5, d1, filters)

    u7 = tf.keras.layers.UpSampling2D(size=2)(u6)
    output = tf.keras.layers.Conv2D(instruments, kernel_size=4, strides=1,
                               padding='same', activation='tanh')(u7)  # 32, 128, 4

    generator = tf.keras.models.Model([c_input, z_input], output, name='Generator')

    return generator

이제 각 레이어(layer)의 입력/출력을 보기 위해, generator의 각 레이어를 살펴 보도록 하겠습니다.

In [ ]:
# Models
generator = build_generator()
generator.summary()

### Critic (Discriminator)

Critic의 목표는 생성된 piano roll이 얼마나 현실적인지 생성기에 피드백을 제공하여, Generator가 보다 현실적인 데이터를 생성하는 방법을 배울 수 있도록 하는 것입니다. Critic은 piano roll이 얼마나 "실제" 또는 "가짜"인지를 나타내는 스칼라(scalar)를 출력하여 이 피드백을 제공합니다.

Critic은 데이터를 "실제" 또는 "가짜"로 분류하려고 시도하기 때문에, 일반적으로 사용되는 이진 분류기(binary classifier)와 크게 다르지 않습니다. 
Critic은 4개의 convolution 레이어와 마지막 계층에 dense 레이어로 구성된 간단한 아키텍처를 사용합니다.

<img src="images/ddis.png" alt="Discriminator architecture" width="800">

In [ ]:
def _build_critic_layer(layer_input, filters, f_size=4):
    """
    This layer decreases the spatial resolution by 2:

        input:  [batch_size, in_channels, H, W]
        output: [batch_size, out_channels, H/2, W/2]
    """
    d = tf.keras.layers.Conv2D(filters, kernel_size=f_size, strides=2,
                               padding='same')(layer_input)
    # Critic does not use batch-norm
    d = tf.keras.layers.LeakyReLU(alpha=0.2)(d) 
    return d


def build_critic(pianoroll_shape=(32, 128, 4), filters=64):
    """WGAN critic."""
    
    condition_input_shape = (32,128,1)
    groundtruth_pianoroll = tf.keras.layers.Input(shape=pianoroll_shape)
    condition_input = tf.keras.layers.Input(shape=condition_input_shape)
    combined_imgs = tf.keras.layers.Concatenate(axis=-1)([groundtruth_pianoroll, condition_input])


    
    d1 = _build_critic_layer(combined_imgs, filters)
    d2 = _build_critic_layer(d1, filters * 2)
    d3 = _build_critic_layer(d2, filters * 4)
    d4 = _build_critic_layer(d3, filters * 8)

    x = tf.keras.layers.Flatten()(d4)
    logit = tf.keras.layers.Dense(1)(x)

    critic = tf.keras.models.Model([groundtruth_pianoroll,condition_input], logit,
                                          name='Critic')
    

    return critic

In [ ]:
# Create the Discriminator

critic = build_critic()
critic.summary() # View discriminator architecture.

## Training

목적 함수(objective function)를 최적화하는 모델 파라메터를 검색하여 모델을 학습합니다. WGAN-GP에는 generator와 critic 네트워크를 번갈아 가며 교대 학습할 때, 목적 함수를 최소화하는 특수 loss function을 사용합니다.

*Generator Loss:*
* Critic Loss 함수의 음수인 Wasserstein (Generator) loss function을 사용합니다. Generator는 생성된 piano roll을 가능한 한 실제 피아노 롤에 최대한 가깝게 하도록 학습됩니다.
    * $\frac{1}{m} \sum_{i=1}^{m} -D_w(G(z^{i}|c^{i})|c^{i})$

*Critic Loss:*

* 먼저 실제 piano roll 확률 분포와 생성된(가짜) piano roll 확률 분포 사이의 거리를 최대화하도록 설계된 Wasserstein (Critic) loss function을 적용합니다.

    * $\frac{1}{m} \sum_{i=1}^{m} [D_w(G(z^{i}|c^{i})|c^{i}) - D_w(x^{i}|c^{i})]$

* 입력 데이터에 대한 critic의 gradient가 어떻게 행동하는지 제어하기 위해 고안된 gradient penalty loss function 항을 추가합니다. 이것은 Generator의 최적화를 더 쉽게 합니다.
    * $\frac{1}{m} \sum_{i=1}^{m}(\lVert \nabla_{\hat{x}^i}D_w(\hat{x}^i|c^{i}) \rVert_2 -  1)^2 $

In [ ]:
# Define the different loss functions

def generator_loss(critic_fake_output):
    """ Wasserstein GAN loss
    (Generator)  -D(G(z|c))
    """
    return -tf.reduce_mean(critic_fake_output)


def wasserstein_loss(critic_real_output, critic_fake_output):
    """ Wasserstein GAN loss
    (Critic)  D(G(z|c)) - D(x|c)
    """
    return tf.reduce_mean(critic_fake_output) - tf.reduce_mean(
        critic_real_output)


def compute_gradient_penalty(critic, x, fake_x):
    
    c = tf.expand_dims(x[..., 0], -1)
    batch_size = x.get_shape().as_list()[0]
    eps_x = tf.random.uniform(
        [batch_size] + [1] * (len(x.get_shape()) - 1))  # B, 1, 1, 1, 1
    inter = eps_x * x + (1.0 - eps_x) * fake_x

    with tf.GradientTape() as g:
        g.watch(inter)
        disc_inter_output = critic((inter,c), training=True)
    grads = g.gradient(disc_inter_output, inter)
    slopes = tf.sqrt(1e-8 + tf.reduce_sum(
        tf.square(grads),
        reduction_indices=tf.range(1, grads.get_shape().ndims)))
    gradient_penalty = tf.reduce_mean(tf.square(slopes - 1.0))
    
    return gradient_penalty


Loss function이 정의된 상태에서 적절한 모델 파라메터 셋을 검색하는 방법을 정의하기 위해 Tensorflow optimizers 클래스를 사용합니다. 본 튜토리얼에서는 일반적으로 사용되는 범용 최적화 기법인 *Adam* 알고리즘을 사용합니다. 또한 학습 시 진행 상황을 저장하기 위한 체크포인트(Checkpoint)도 설정합니다.

In [ ]:
# Setup Adam optimizers for both G and D
generator_optimizer = tf.keras.optimizers.Adam(1e-3, beta_1=0.5, beta_2=0.9)
critic_optimizer = tf.keras.optimizers.Adam(1e-3, beta_1=0.5, beta_2=0.9)

# We define our checkpoint directory and where to save trained checkpoints
ckpt = tf.train.Checkpoint(generator=generator,
                           generator_optimizer=generator_optimizer,
                           critic=critic,
                           critic_optimizer=critic_optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, check_dir, max_to_keep=5)

이제 `generator_train_step` 및 `critic_train_step` 함수를 정의합니다. 각 함수는 배치에서 단일 forward pass를 수행하고 해당 loss를 리턴합니다.

In [ ]:
@tf.function
def generator_train_step(x, condition_track_idx=0):

    ############################################
    #(1) Update G network: maximize D(G(z|c))
    ############################################

    # Extract condition track to make real batches pianoroll
    c = tf.expand_dims(x[..., condition_track_idx], -1)

    # Generate batch of latent vectors
    z = tf.random.truncated_normal([BATCH_SIZE, 2, 8, 512])

    with tf.GradientTape() as tape:
        fake_x = generator((c, z), training=True)
        fake_output = critic((fake_x,c), training=False)

        # Calculate Generator's loss based on this generated output
        gen_loss = generator_loss(fake_output)

    # Calculate gradients for Generator
    gradients_of_generator = tape.gradient(gen_loss,
                                           generator.trainable_variables)
    # Update Generator
    generator_optimizer.apply_gradients(
        zip(gradients_of_generator, generator.trainable_variables))

    return gen_loss


In [ ]:
@tf.function
def critic_train_step(x, condition_track_idx=0):

    ############################################################################
    #(2) Update D network: maximize (D(x|c)) + (1 - D(G(z|c))|c) + GradientPenality() 
    ############################################################################

    # Extract condition track to make real batches pianoroll
    c = tf.expand_dims(x[..., condition_track_idx], -1)

    # Generate batch of latent vectors
    z = tf.random.truncated_normal([BATCH_SIZE, 2, 8, 512])

    # Generated fake pianoroll
    fake_x = generator((c, z), training=False)


    # Update critic parameters
    with tf.GradientTape() as tape:
        real_output = critic((x,c), training=True)
        fake_output = critic((fake_x,c), training=True)
        critic_loss =  wasserstein_loss(real_output, fake_output)

    # Caculate the gradients from the real and fake batches
    grads_of_critic = tape.gradient(critic_loss,
                                               critic.trainable_variables)

    with tf.GradientTape() as tape:
        gp_loss = compute_gradient_penalty(critic, x, fake_x)
        gp_loss *= 10.0

    # Calculate the gradients penalty from the real and fake batches
    grads_gp = tape.gradient(gp_loss, critic.trainable_variables)
    gradients_of_critic = [g + ggp for g, ggp in
                                  zip(grads_of_critic, grads_gp)
                                  if ggp is not None]

    # Update Critic
    critic_optimizer.apply_gradients(
        zip(gradients_of_critic, critic.trainable_variables))

    return critic_loss + gp_loss


학습을 시작하기 전에 몇 가지 학습 설정 파라메터들를 정의하고 주요 정량적 지표들을 모니터링할 준비를 하겠습니다. 여기에서 학습 중단시기를 결정하는 데 사용할 수 있는 loss 및 metric을 기록합니다. 이 파라메터들을 조정하고 모델이 어떻게 반응하는지 알아 보려면 이 코드 셀(Code shell)로 돌아와 주세요.

In [ ]:
# We use load_melody_samples() to load 10 input data samples from our dataset into sample_x 
# and 10 random noise latent vectors into sample_z
sample_x, sample_z = inference_utils.load_melody_samples(n_sample=10)

In [ ]:
# Number of iterations to train for
iterations = 1000

# Update critic n times per generator update 
n_dis_updates_per_gen_update = 5

# Determine input track in sample_x that we condition on
condition_track_idx = 0 
sample_c = tf.expand_dims(sample_x[..., condition_track_idx], -1)

이제 모델을 학습해 보겠습니다!

In [ ]:
# Clear out any old metrics we've collected
metrics_utils.metrics_manager.initialize()

# Keep a running list of various quantities:
c_losses = []
g_losses = []

# Data iterator to iterate over our dataset
it = iter(dataset)

for iteration in range(iterations):

    # Train critic
    for _ in range(n_dis_updates_per_gen_update):
        c_loss = critic_train_step(next(it))

    # Train generator
    g_loss = generator_train_step(next(it))

    # Save Losses for plotting later
    c_losses.append(c_loss)
    g_losses.append(g_loss)

    display.clear_output(wait=True)
    fig = plt.figure(figsize=(15, 5))
    line1, = plt.plot(range(iteration+1), c_losses, 'r')
    line2, = plt.plot(range(iteration+1), g_losses, 'k')
    plt.xlabel('Iterations')
    plt.ylabel('Losses')
    plt.legend((line1, line2), ('C-loss', 'G-loss'))
    display.display(fig)
    plt.close(fig)
    
    # Output training stats
    print('Iteration {}, c_loss={:.2f}, g_loss={:.2f}'.format(iteration, c_loss, g_loss))
    
    # Save checkpoints, music metrics, generated output
    if iteration < 100 or iteration % 50 == 0 :
        # Check how the generator is doing by saving G's samples on fixed_noise
        fake_sample_x = generator((sample_c, sample_z), training=False)
        metrics_utils.metrics_manager.append_metrics_for_iteration(fake_sample_x.numpy(), iteration)

        if iteration % 50 == 0:
            # Save the checkpoint to disk.
            ckpt_manager.save(checkpoint_number=iteration) 
        
            fake_sample_x = fake_sample_x.numpy()
    
            # plot the pianoroll
            display_utils.plot_pianoroll(iteration, sample_x[:4], fake_sample_x[:4], save_dir=train_dir)

            # generate the midi
            destination_path = path_utils.generated_midi_path_for_iteration(iteration, saveto_dir=sample_dir)
            midi_utils.save_pianoroll_as_midi(fake_sample_x[:4], destination_path=destination_path)


###  이제 학습을 시작했습니다!

Wasserstein loss function을 사용할 때, 여러분은 critic을 수렴하도록(generator 업데이트의 gradient가 정확한 지의 여부를 확인) 학습시켜야 합니다. 이는 vanishing gradient를 피하기 위해 critic이 너무 강해지지 않도록 하는 것이 중요한 표준 GAN과는 대조적입니다.

따라서 Wasserstein loss를 사용하면 GAN 학습의 주요 어려움들 중 하나인 discriminator와 generator 학습의 균형을 맞추는 방법이 제거됩니다. WGAN을 사용하면 generator 업데이트 간에 여러 번 critic을 학습시켜 수렴에 가깝게 만들 수 있습니다. 일반적으로 사용되는 비율은 1번 generator 업데이트 시, 5번의 critic 업데이트를 수행하는 것입니다.


### 학습 과정 모니터링

이러한 모델을 학습 시 시간과 리소스가 많이 소요되므로, 예외가 발생하는 경우 이상점들(anomalies)을 포착하고 해결하기 위해 지속적으로 학습 과정을 모니터링해야 합니다. 주의해야 할 사항들은 다음과 같습니다.

**Loss는 어떻게 보이나요?**

Adversarial 학습 과정은 매우 역동적이고 고주파 진동 현상이 매우 일반적입니다. 그러나 Loss(critic 또는 generator)가 큰 값으로 급등하거나 0으로 급락하거나 단일 값에 정체되어 있으면, 어딘가에 문제가 있을 수 있습니다.

**모델이 정상적으로 학습 중인가요?**

- 가능한 경우, Critic loss 및 기타 음악 품질 지표들을 모니터링합니다. 예상 궤도를 따르고 있나요?
- 생성된 샘플들(piano rolls)을 모니터링합니다. 이들이 시간이 지남에 따라 개선되고 있나요? mode collapse 현상이 보이나요? 샘플들을 직접 들어 보셨나요? 

**언제 중지해야 하는지 어떻게 알 수 있나요?**

- 샘플이 기대치를 충족시키는 경우
- Critic loss가 더 이상 개선되지 않을 경우
- 음악 품질 지표의 기대값(expected value)이 학습 데이터에 대한 동일한 지표의 해당 기대값에 수렴 시

### 학습 중 샘플 품질을 측정하는 방법

일반적으로 모든 종류의 신경망을 학습할 때는 학습 기간 동안 loss function 값을 모니터링하는 것이 표준 관행입니다. WGAN의 critic loss은 샘플 품질과 밀접한 관련이 있는 것으로 밝혀졌습니다.

분류 또는 회귀와 같은 보다 전통적인(traditional) 모델의 정확도를 평가하기 위한 표준 메커니즘이 존재하지만, generative 모델을 평가하는 것은 활발한 연구 분야이며 음악 생성 도메인 내에서는 더 어렵습니다.

이 문제를 해결하기 위해, 데이터에 대한 높은 수준의 측정들을 수행하고 모델이 이러한 측정치들에 맞는 음악을 얼마나 잘 생성하는지 확인해 보겠습니다. 만약 어려분의 모델이 학습 데이터셋에 대해 이러한 측정의 평균값(mean value)에 가까운 음악을 생성한다면, 음악은 일반적인 “모양(shape)” 과 일치해야 합니다.

다음과 같은 세 가지 측정들을 살펴 보겠습니다.

- **Empty bar rate:** 총 bar 대비 empty bar의 비율입니다.
- **Pitch histogram distance:** 피치(pitch)의 분포와 위치를 캡처하는 지표입니다.
- **In Scale Ratio:** 음악에서 발견되는 공통 키인 C 메이저 키에 있는 노트 수와 총 노트 수의 비율입니다.

## Evaluate results

학습이 완료되었으니 모델을 평가해 봅시다. 여러분은 다양한 방법들로 모델을 분석할 수 있습니다.

1. 학습하는 동안 generator와 critic 손실이 어떻게 변했는지 조사
2. 학습하는 동안 특정 음악 지표가 어떻게 변경되었는지 분석
3. 매 iteration마다 고정 입력값을 위해 생성된 piano roll 출력 시각화 및 비디오 생성

마지막으로 저장된 체크포인트를 먼저 복원하겠습니다. 학습을 완료하지 않았지만 사전 학습된(pre-trained) 버전으로 계속하려면 `TRAIN = False`로 설정하세요.

In [ ]:
ckpt = tf.train.Checkpoint(generator=generator)
ckpt_manager = tf.train.CheckpointManager(ckpt, check_dir, max_to_keep=5)

ckpt.restore(ckpt_manager.latest_checkpoint).expect_partial()
print('Latest checkpoint {} restored.'.format(ckpt_manager.latest_checkpoint))

### Plot losses

In [ ]:
display_utils.plot_loss_logs(g_losses, c_losses, figsize=(15, 5), smoothing=0.01)

학습하면서 critic loss(그래프의 C_loss)가 어떻게 0으로 감소하는지 관찰해 보세요. WGAN-GP에서는 학습할 때 critic loss가 거의 단조(monotonically) 감소합니다.

### Plot metrics

In [ ]:
metrics_utils.metrics_manager.set_reference_metrics(training_data)
metrics_utils.metrics_manager.plot_metrics()

여기의 각 행은 서로 다른 음악 품질 지표에 해당하며 각 열은 악기 트랙(instrument track)을 나타냅니다.

Iteration 횟수가 증가함에 따라 서로 다른 지표의 기대값(파란색 scatter)이 해당 학습셋의 기대값(빨간색)에 어떻게 가까워지는지 관찰해 보세요. 모델이 수렴함에 따라 리턴값이 감소함을 기대할 수 있습니다.

### 학습 중 생성된 샘플
아래 Code cell은 학습 과정에서 생성된 중간 샘플들을 조사하는 데 도움이 됩니다. 여기서 조건부 입력(conditioned input)은 학습 데이터에서 샘플링됩니다. Iteration 0과 iteration 100에서 샘플을 듣고 관찰하는 것으로 시작하겠습니다. 차이점을 주목해 주세요!

In [ ]:
# Enter an iteration number (can be divided by 50) and listen to the midi at that iteration
iteration = 50
midi_file = os.path.join(sample_dir, 'iteration-{}.mid'.format(iteration))
display_utils.playmidi(midi_file)    

In [ ]:
# Enter an iteration number (can be divided by 50) and look at the generated pianorolls at that iteration
iteration = 50
pianoroll_png = os.path.join(train_dir, 'sample_iteration_%05d.png' % iteration)
display.Image(filename=pianoroll_png)

생성된 piano roll이 iteration 횟수에 따라 어떻게 변하는 지 확인해 봅니다.

In [ ]:
from IPython.display import Video

display_utils.make_training_video(train_dir)
video_path = "movie.mp4"
Video(video_path)

## Inference 

### Generating accompaniment for custom input

축하합니다! 음악을 생성하도록 사용자 정의 WGAN-GP를 학습했습니다. Generator가 사용자 정의 입력(custom input)에서 어떻게 수행되는지 확인해 보세요.
아래 Code cell은 "Twinkle Twinkle Little Star"를 기반으로 새 노래를 생성합니다.

In [ ]:
latest_midi = inference_utils.generate_midi(generator, eval_dir, input_midi_file='./input_twinkle_twinkle.mid')

In [ ]:
display_utils.playmidi(latest_midi)

또한 특정 샘플에 대해 생성된 piano roll들을 보고 얼마나 다양한 지 확인할 수 있습니다!

In [ ]:
inference_utils.show_generated_pianorolls(generator, eval_dir, input_midi_file='./input_twinkle_twinkle.mid')

# What's next?

### (Optional) 사용자 정의 데이터 사용

사용자 정의 데이터셋을 만들려면 MIDI 데이터에서 piano roll을 추출할 수 있습니다. MIDI 파일에서 pinao roll을 만드는 예는 다음과 같습니다.

In [ ]:
import numpy as np
from pypianoroll import Multitrack

midi_data = Multitrack('./input_twinkle_twinkle.mid')
tracks = [track.pianoroll for track in midi_data.tracks]
sample = np.stack(tracks, axis=-1)

print(sample.shape)

# Appendix

### 오픈 소스 구현
음악에 대한 generative 모델의 오픈 소스 구현에 대해서는 아래 링크들을 확인하세요.

- [MuseGAN](https://github.com/salu133445/musegan): GAN을 사용하여 멀티 트랙 폴리포닉(polyphonic) 음악을 생성하는 공식 TensorFlow 구현
- [GANSynth](https://github.com/tensorflow/magenta/tree/master/magenta/models/gansynth): 프로그레시브 GAN 아키텍처를 사용하여 단일 벡터에서 전체 오디오 스펙트로그램(spectrogram)으로 컨볼루션을 사용하여 업샘플링
- [Music Transformer](https://github.com/tensorflow/magenta/tree/master/magenta/models/score2perf): 트랜스포머(Transformer)를 사용하여 음악 생성

GAN은 또한 도메인 간 이미지 이동, 유명인사 얼굴 생성, 이미지에 대한 초고해상도 텍스트, 이미지 inpainting을 포함한 여러 도메인에서 성과를 보이고 있습니다.

- [Keras-GAN](https://github.com/eriklindernoren/Keras-GAN): 이미지 생성을 위한 Keras의 참조 구현 라이브러리(교육 목적에 적합).

여러 분야들에 대한 확률 분포를 모델링하기 위해 GAN을 사용하는 문헌들이 많이 있습니다! 관심이 있으시면 [Gan Zoo](https://github.com/hindupuravinash/the-gan-zoo)에서 시작해 보세요.

### References
<a id='references'></a>
1. [Dong, H.W., Hsiao, W.Y., Yang, L.C. and Yang, Y.H., 2018, April. MuseGAN: Multi-track sequential generative adversarial networks for symbolic music generation and accompaniment. In Thirty-Second AAAI Conference on Artificial Intelligence.](https://arxiv.org/abs/1709.06298)
2. [Ishaan, G., Faruk, A., Martin, A., Vincent, D. and Aaron, C., 2017. Improved training of wasserstein gans. In Advances in Neural Information Processing Systems.](https://arxiv.org/abs/1704.00028)
3. [Arjovsky, M., Chintala, S. and Bottou, L., 2017. Wasserstein gan. arXiv preprint arXiv:1701.07875.](https://arxiv.org/abs/1701.07875)
4. [Foster, D., 2019. Generative Deep Learning: Teaching Machines to Paint, Write, Compose, and Play. O'Reilly Media.](https://www.amazon.com/Generative-Deep-Learning-Teaching-Machines/dp/1492041947)

### (Optional) Wassertein GAN with Gradient Penalty에 대한 보충 내용

GAN은 generative 모델링을 위한 주요 혁신이지만, 일반(plain) GAN도 학습하기가 어렵습니다. 몇 가지 일반적인 문제들은 다음과 같습니다.

* **Oscillating loss:** Discriminator와 generator의 loss는 장기적인 안정성을 나타내지 않고 진동하기 시작할 수 있습니다.
* **Mode collapse:** Generator는 항상 discriminator를 속이는 작은 샘플셋에 갇힐 수 있습니다. 이것은 새로운 샘플을 생성하는 네트워크의 능력을 감소시킵니다.
* **Uninformative loss:** Generator loss와 생성된 출력 데이터의 품질 사이에 상관 관계가 없기 때문에, 일반 GAN 학습을 해석하기가 어렵습니다.

[Wasserstein GAN](#references)은 GAN의 주요 발전으로 이러한 문제들 중 일부를 완화하는 데 도움이 되었습니다. 그 기능들 중 일부는 다음과 같습니다.

1. Loss function의 해석성을 크게 개선하고 보다 명확한 학습 중단 기준(stopping criteria)을 제공합니다.
2. WGAN은 일반적으로 더 고품질의 결과를 생성하며, 이는 이미지 생성 도메인에서 실험으로 확인되었습니다.

**Wasserstein GAN with Gradient Penalty의 수학적 내용**

실제 분포 $P_r$와 생성된 piano roll 분포 $P_g$ 사이의 [Wasserstein distance](https://en.wikipedia.org/wiki/Wasserstein_metric)는 다음과 같이 정의됩니다.

$$\mathbb{W}(P_{r},P_{g})=\sup_{\lVert{f} \rVert_{L} \le 1} \mathbb{E}_{x \sim \mathbb{P}_r}(f(x)) - \mathbb{E}_{x \sim \mathbb{P}_g}(f(x)) $$

이 수식에서 우리는 실제 분포의 기대값과 생성 분포의 기대값 사이의 거리를 최소화하려고 합니다. 이 때, $f$는 [1-Lipschitz](https://en.wikipedia.org/wiki/Lipschitz_continuity) 여야 한다는 기술적 제약이 따릅니다.

Gradient가 너무 빠르게 변하는 것을 기본적으로 제한하는 1-Lipschitz 조건을 강제하기 위해 gradient penalty를 사용합니다.

**Gradient penalty** : 우리는 critic의 gradient에 핸디캡을 부여하고 싶습니다. 데이터 분포 $P_r$와 생성자 분포 $P_g$에서 샘플링된 점 쌍(pairs of points) 사이의 직선을 따라 균일하게 샘플링하여 $P_{\hat{x}}$를 암시적으로(implicitly) 정의합니다. 이것은 최적의 critic이 $P_r$와 $P_g$의 결합 점을 연결하는 gradient norm 1을 가진 직선을 포함한다는 사실에 의해 모티베이션을 받은 것입니다. 본 구현에서는 원래 논문에서 권장한 penalty coefficient $\lambda = 10$을 사용합니다.

Gradient penalty로 인한 loss는 다음과 같습니다.

$$\mathbb{L}(P_{r},P_{g},P_{\hat{x}} )= \mathbb{W}(P_{r},P_{g}) + \lambda \mathbb{E}_{\hat{x} \sim \mathbb{P}_\hat{x}}[(\lVert \nabla_{\hat{x}}D(\hat{x}) \rVert_2 -  1)^2]$$

이 loss는 $w$ 및 $\theta$로 매개변수화할 수 있습니다. 그런 다음, 신경망을 사용하여 $f_w$ (discriminator) 와 $g_\theta$ (generator) functrion을 학습합니다.

$$\mathbb{W}(P_{r},P_{\theta})=\max_{w \in \mathbb{W}} \mathbb{E}_{x \sim \mathbb{P}_r}(D_w(x)) - \mathbb{E}_{z \sim p(z)}(D_w(G_{\theta}(z)) $$
$$\mathbb{L}(P_{r},P_{\theta},P_{\hat{x}})=\max_{w \in \mathbb{W}} \mathbb{E}_{x \sim \mathbb{P}_r}(D_w(x)) - \mathbb{E}_{z \sim p(z)}(D_w(G_{\theta}(z)) + \lambda \mathbb{E}_{\hat{x} \sim \mathbb{P}_\hat{x}}[(\lVert \nabla_{\hat{x}}D_w(\hat{x}) \rVert_2 -  1)^2]$$

이 때, $\hat{x}$과 $\epsilon$은 아래와 같습니다. $$ \hat{x} = \epsilon x + (1- \epsilon) G(z), \;\; \epsilon \sim Unif(0,1)$$


학습 기본 절차는 다음과 같습니다.
1. 실제 분포 $P_r$에서 real_x를 추출하고 생성된 분포 $G_{\theta}(z)$에서 fake_x를 추출합니다. ($z \sim p(z)$)
2. z에서 latent vector들을 샘플링한 다음, $G_{\theta}$ generator로 변환하여 가짜 샘플 fake_x를 얻습니다. 변환된 샘플들은 Critic function $D_w$에 의해 평가됩니다.
3. 두 분포 사이의 Wasserstein distance를 최소화합니다.

generator와 critic은 모두 입력 piano roll 멜로디에서 조절됩니다.